In [2]:
import os
import random
import torch
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer

lines = {}
with open("movie_lines.txt", encoding="ISO-8859-1") as f:
    for line in f:
        p = line.strip().split(" +++$+++ ")
        if len(p) == 5: lines[p[0]] = p[4]

convs = []
with open("movie_conversations.txt", encoding="ISO-8859-1") as f:
    for line in f:
        p = line.strip().split(" +++$+++ ")
        if len(p) == 4: convs.append(eval(p[3]))

pairs = []
for c in convs:
    for i in range(len(c) - 1):
        if c[i] in lines and c[i+1] in lines:
            q, a = lines[c[i]].strip(), lines[c[i+1]].strip()
            if 3 < len(q.split()) < 25 and 3 < len(a.split()) < 25:
                pairs.append({"input": q, "output": a})

model_id = "microsoft/DialoGPT-medium"
tk = AutoTokenizer.from_pretrained(model_id)
tk.pad_token = tk.eos_token
data = Dataset.from_list(random.sample(pairs, 5000))

def tokenize(item):
    txt = item["input"] + tk.eos_token + item["output"] + tk.eos_token
    res = tk(txt, truncation=True, padding="max_length", max_length=64)
    res["labels"] = res["input_ids"].copy()
    return res

tokenized_data = data.map(tokenize)
model = AutoModelForCausalLM.from_pretrained(model_id)

args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    num_train_epochs=2,
    fp16=True,
    report_to="none"
)

trainer = Trainer(model=model, args=args, train_dataset=tokenized_data, tokenizer=tk)
trainer.train()

trainer.save_model("./final_model")
tk.save_pretrained("./final_model")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

...

model.safetensors:   0%|          | 0.00/863M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

/tmp/ipython-input-2959653220.py:50: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(model=model, args=args, train_dataset=tokenized_data, tokenizer=tk)
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 50256}.
`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
500,1.666700
1000,1.423500


('./final_model/tokenizer_config.json',
 './final_model/special_tokens_map.json',
 './final_model/chat_template.jinja',
 './final_model/vocab.json',
 './final_model/merges.txt',
 './final_model/added_tokens.json',
 './final_model/tokenizer.json')